Goal: Allow the user to create a directory of small images (of the same size?) and use them as a data set to mask an image.  This example code will try to find the closest match based on RGB similarties between image region and sample images.

In [1]:
from PIL import Image
from os import listdir
from IPython.core.display import display, HTML
# NOTE: update to work with different image source
#emojiCharSet_24x24_35.png

In [2]:
# Read average RGB values for this image
def imageRGBs( image ):
    #image = 'files/imageset/csharp.png'
    im = Image.open(image)
    width, height = im.size
    pix = im.load()
    r = 0
    g = 0
    b = 0
    count = width*height
    for w in range(width):
        for h in range(height):
            r += pix[w,h][0]
            g += pix[w,h][1]
            b += pix[w,h][2]
    print(image,(int(r/count), int(g/count),int(b/count)))
    return (image,(int(r/count), int(g/count),int(b/count)))

In [12]:
def makeCharImageSet( directory, height ):
    '''inputs: directory of images, height in pixels of output imageset
    outputs: filename of output imageset'''
    width = height*len(listdir(directory))
    # Create shell of resulting image
    resultImg = Image.new('RGB', (width,height))
    count = 0
    for image in listdir(directory):
        path = directory + '/' + image
        im = Image.open( path )
        resultImg.paste(im, (count*height,0))
        count += 1
    resultImg.save('charImageSet.jpg','JPEG')
    display(HTML('<img src="charImageSet.jpg">'))
    return 'charImageSet.jpg'

In [15]:
def get_RGB_dictionary(imageset, size):
    '''inputs: image containing uniform sized icons, size in pixels of each icon
    outputs: a list of icon index and average RGB values'''
    results = []  # store image index and average RGB tuples (1, (0,0,0))

    im = Image.open(imageset)
    width, height = im.size
    pix = im.load()
    index = 0
    for box in range(0,width,size):   # 0, 48, 96... moving x starting place
        r = 0
        g = 0
        b = 0
        for w in range(box, box+size):
            for h in range(height):
                r += pix[w,h][0]
                g += pix[w,h][1]
                b += pix[w,h][2]
        avgR = r/(size**2)
        avgG = g/(size**2)
        avgB = b/(size**2)
        results.append( (index, (avgR,avgG,avgB)) )
        index += 1
    return results

In [33]:
# find an image to convert to an alternate image set.  open this image and scale to a certain % size
def scaleInputImage( image_path, width_multiple, height_multiple, scale):
    im = Image.open(image_path)
    print(im.width, im.height)
    print(im.width-im.width%width_multiple,im.height-im.height%height_multiple)
    imresize = im.resize(((im.width-im.width%width_multiple)*scale, (im.height-im.height%height_multiple)*scale), Image.ANTIALIAS)

    newname = image_path.split('.')[0] + '_%dx%d.png' %(width_multiple,height_multiple)
    imresize.save(newname)
    return newname

In [30]:
def findClosestRGB( testTuple, replaceList ):
    # testTuple in form (avgR, avgG, argB)
    # replaceList in form [ (index, (avgR, avgG, avgB)), ... ]
    minDiff = 3*255
    minIndex = 0
    for icon in replaceList:
        idx = icon[0]
        iR = icon[1][0]
        iG = icon[1][1]
        iB = icon[1][2]
        diff = abs(testTuple[0]-iR) + abs(testTuple[1]-iG) + abs(testTuple[2]-iB)
        if diff < minDiff:
            minIndex = idx
            minDiff = diff
    return minIndex

In [48]:
# input arguments:
imageDirectory = 'imageset'
size = 48
testImage = 'test.jpg'

# STEP 1: Create a single image set out of a directory of sample images.
replaceImage = makeCharImageSet(imageDirectory, size)
print('(1) replacement imageset created:', replaceImage)

# STEP 2: Scan through the replaceImage file and create a list of
#         replaceImage avgerage RGB values
replaceRGBlist = get_RGB_dictionary(replaceImage, size)
print('(2) list of replacment RGB values created: %d items' %(len(replaceRGBlist)))
#print(replaceRGBlist,'\n')
    
# STEP 3: Scale test image so that it has %size dimensions
# TODO: update algorithm below so that this isn't required
testImage = scaleInputImage(testImage, size, size, 4)
print('(3) test image scaled. ready to be processed:', testImage)

# STEP 4: 
# scan through the scaled test image in chuncks of 48x48 (or custom) and find the average RGB for each region
# next, compare that average RGB against the test set to find the closest match.
# finally, replace this image with the test set image.
 
im = Image.open(testImage)
pix = im.load()

replaceImage = Image.open(replaceImage)
#charpix = charimg.load()

for boxX in range(0,im.width,size):
    for boxY in range(0,im.height,size):
        # loop over each box
        r = 0
        g = 0
        b = 0
        for x in range(boxX,boxX+size):
            for y in range(boxY, boxY+size):
                r += pix[x,y][0]
                g += pix[x,y][1]
                b += pix[x,y][2]
        # average the r,g,b pixel values
        rAvg = r/(size*size)
        gAvg = g/(size*size)
        bAvg = b/(size*size)
        avgRGB = (rAvg, gAvg, bAvg)
        # next, compare this tuple of averaged RGB values
        # against the averaged RGB tuples in the replacement image set
        # the method will return the location in the replacement image
        # set with the minimum difference
        
        index = findClosestRGB( avgRGB, replaceRGBlist )
        #print(avgRGB,'index',index)
        #top,bot = topBot(pix, i, j, 48, 48)
        
        #index = picChar(_charDict, (top,bot))
        
        chop = replaceImage.crop((index*size,0,index*size+size,size))
        im.paste(chop, (boxX,boxY,boxX+size,boxY+size))
        
im.save('result.png')
print('Done.')






(1) replacement imageset created: charImageSet.jpg
(2) list of replacment RGB values created: 31 items
1180 842
1152 816
(3) test image scaled. ready to be processed: test_48x48.png
Done.
